# Boto3

## Connect

In [ ]:
import boto3

client = boto3.client(
    's3',
    endpoint_url='https://s3.hpccloud.lngs.infn.it/',
    aws_access_key_id='JIUG9PGAKP2ULWP3WQ22',
    aws_secret_access_key='VLynhYvsV9JXGAqVxAMWdmo7X1THDB2p2lwxX7uy',
)

## List all in a bucket

In [ ]:
bucket_name = 'prova.ifab'

In [ ]:
paginator = client.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=''):
    for obj in page.get('Contents', []):
        print(obj['Key'])

## Read a file

### Geo Dataframe

In [ ]:
import geopandas as gpd
from io import BytesIO

bucket_name = 'ifab.data'
file_key = 'locations/addresses_IFAB.geojson'

response = client.get_object(Bucket=bucket_name, Key=file_key)
gdf = gpd.read_file(BytesIO(response['Body'].read()))

In [ ]:
gdf

### Image

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

bucket_name = 'prova.ifab'
file_key = 'immagini/output.png'

response = client.get_object(Bucket=bucket_name, Key=file_key)

# Apri con PIL
img = Image.open(BytesIO(response['Body'].read()))

# Mostra con matplotlib
plt.imshow(img)
plt.axis('off')
plt.show()

## Crea bucket

In [ ]:
client.create_bucket(Bucket='prova.ifab')

## Upload a file

In [ ]:
bucket_name = 'prova.ifab'

### Geo Dataframe

In [ ]:
# Inserisce in memoria il geodaframe
buffer = BytesIO()
gdf.to_file(buffer, driver='GeoJSON')

# Carica sul bucket
buffer.seek(0)
client.upload_fileobj(buffer, bucket_name, 'addresses_IFAB.geojson')

### Image

In [ ]:
from PIL import Image
from io import BytesIO

# Immagine PIL (creata o elaborata)
img = Image.new('RGB', (100, 100), color='red')

# Salva in un buffer
img_buffer = BytesIO()
img.save(img_buffer, format='PNG')  # o 'JPEG', ecc.

# Upload
img_buffer.seek(0)
client.upload_fileobj(img_buffer, bucket_name, 'immagini/output.png')

## List our bucket

In [ ]:
client.list_buckets()

## Delete a file

In [ ]:
bucket_name = 'prova.ifab'
file_key = 'addresses_IFAB.geojson'

client.delete_object(Bucket=bucket_name, Key=file_key)